In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
# Cria o YAML com os caminhos corretos
yaml_content = """
train: /kaggle/input/anotacoes/Anotacoes YOLO/train/images
val: /kaggle/input/anotacoes/Anotacoes YOLO/val/images
test: /kaggle/input/anotacoes/Anotacoes YOLO/test/images

nc: 3
names: ['watertank', 'pool', 'waste']
"""

# Salva o arquivo no diretório atual
with open("custom_data.yaml", "w") as f:
    f.write(yaml_content)


In [ ]:
results = model.train(data="custom_data.yaml", epochs=50, imgsz=256)

In [ ]:
best_model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

In [ ]:
from ultralytics import YOLO
from PIL import Image  # Importar PIL.Image para fromarray
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
import re
import math
import glob

In [ ]:
!pip install opencv-python

In [ ]:
# Função para converter tile em latitude e longitude
def tile_to_latlon(zoom, x, y):
    """Converte coordenadas de tile (z, x, y) em latitude e longitude."""
    n = 2.0 ** zoom
    lon_deg = x / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
    lat_deg = math.degrees(lat_rad)
    return lat_deg, lon_deg

# Função para extrair z, x, y do caminho da imagem
def extract_tile_info(image_path):
    """Extrai z, x, y do caminho da imagem."""
    # Exemplo: https://ortofotos.linhares.es.gov.br/SEDE/2022/21/815062/1163306.png
    pattern = r'(\d+)/(\d+)/(\d+)\.png$'
    match = re.search(pattern, image_path)
    if match:
        z, x, y = map(int, match.groups())
        return z, x, y
    else:
        raise ValueError(f"Formato de caminho inválido: {image_path}")

In [ ]:
# Diretório com as imagens
image_dir = '/kaggle/input/inferencia-z21/21/815173/*.png'  # Ajuste o caminho para sua pasta de imagens
image_paths = glob.glob(image_dir)  # Lista todas as imagens PNG na pasta

In [ ]:
import cv2

In [ ]:
# Lista para armazenar os dados
data = []

# Itera sobre todas as imagens
for image_path in image_paths:
    # Realiza a predição
    results = best_model.predict(image_path, conf=0.5, imgsz=256)

    # Obtém a imagem anotada (opcional, para visualização)
    #annotated_img = results[0].plot()
    #annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    #annotated_img_pil = Image.fromarray(annotated_img_rgb)

    # Exibe a imagem (opcional, pode ser removido em produção)
    #plt.figure(figsize=(8, 8))
    #plt.imshow(annotated_img_pil)
    #plt.axis('off')
    #plt.show()

    # Extrai z, x, y do caminho da imagem
    try:
        z, x, y = extract_tile_info(image_path)
        lat, lon = tile_to_latlon(z, x, y)
    except ValueError as e:
        print(e)
        continue

    # Extrai as coordenadas das caixas delimitadoras
    for i, result in enumerate(results[0].boxes):
        box_coords = result.xyxy[0].cpu().numpy()
        x_min, y_min, x_max, y_max = box_coords
        class_id = int(result.cls.cpu().numpy().item())
        class_name = results[0].names[class_id]

        # Imprime os resultados
        print(f'Imagem: {image_path}')
        print(f'Objeto {i+1}:')
        print(f'  Classe: {class_name}')
        print(f'  Coordenadas (x_min, y_min, x_max, y_max): ({x_min:.2f}, {y_min:.2f}, {x_max:.2f}, {y_max:.2f})')
        print(f'  Latitude: {lat:.6f}, Longitude: {lon:.6f}\n')

        # Adiciona os dados à lista
        data.append({
            'image_path': image_path,
            'object_id': i + 1,
            'class': class_name,
            'x_min': x_min,
            'y_min': y_min,
            'x_max': x_max,
            'y_max': y_max,
            'latitude': lat,
            'longitude': lon
        })

# Cria um DataFrame com os dados
df = pd.DataFrame(data)

# Salva o DataFrame em um arquivo CSV
output_csv = '/kaggle/working/coordenadas_com_latlon.csv'
df.to_csv(output_csv, index=False)
print(f"CSV salvo em: {output_csv}")